In [1]:
import requests
from lxml import html
import pandas as pd
from functools import cache

In [36]:
RELEASES_URL = 'https://learn.microsoft.com/en-us/azure/databricks/release-notes/runtime/'

LTS_RELEASES_XPATH = '//table[1]'
ALL_RELEASES_XPATH = '//table[2]'

INSTALLED_JAVA_SCALA_LIBS_XPATH = '//table[3]'
INSTALLED_JAVA_SCALA_LIBS_ML_CPU_XPATH = '//table[3]'
INSTALLED_JAVA_SCALA_LIBS_ML_GPU_XPATH = '//table[4]'

SCALA_VERSION_XPATH = '//li/strong[text()="Scala"]/text()'
DELTA_LAKE_VERSION_XPATH = '//li/strong[text()="Delta Lake"]/text()'

In [20]:
@cache
def load_content(url):
    # with open(
    #         '../tmp/Databricks Runtime release notes versions and compatibility - Azure Databricks _ Microsoft Learn.html',
    #         'r') as f:
    #     return f.read()  #  
        return requests.get(url).content


def extract_from_content(content, xpath):
    return html.tostring(html.fromstring(content).xpath(xpath)[0])


def extract_from_content2(content, xpath):
    return html.fromstring(content).xpath(xpath)


def load_table(url, xpath):
    content = load_content(url)
    table_html = extract_from_content(content, xpath)
    return pd.read_html(table_html)[0]

In [21]:
releases = load_table(RELEASES_URL, ALL_RELEASES_XPATH)

In [31]:
size = len(releases)
releases['spark_version'] = releases['Apache Spark version']
releases['release_date'] = pd.to_datetime(releases['Release date'], format='%b %d, %Y')
releases['end_of_support_date'] = pd.to_datetime(releases['End-of-support date'], format='%b %d, %Y')
releases['version'] = releases['Version'].transform(lambda v: v.split()[0])
releases['is_lts'] = releases['Version'].transform(lambda v: v.upper().endswith('LTS'))
releases['is_ml'] = pd.Series([[True, False]] * size)  # TODO: think of a better way
releases_exploded_tmp = releases.explode('is_ml')
releases_exploded_tmp['is_gpu_accelerated'] = releases_exploded_tmp['is_ml'].apply(
    lambda is_ml: [True, False] if is_ml else None)  # TODO: think of a better way
releases_exploded = releases_exploded_tmp.explode('is_gpu_accelerated')


def build_relative_path(version, is_lts, is_ml, is_gpu_accelerated):
    if is_ml:
        path = f'runtime-{version}{"-lts" if is_lts else ""}' + '/runtime-' + version
    else:
        path = 'runtime-' + version
    if is_lts:
        path += '-lts'
    if is_ml and is_gpu_accelerated:
        path += '-ml-gpu'
    elif is_ml and not is_gpu_accelerated:
        path += '-ml-cpu'
    return path


def build_url(version, is_lts, is_ml):
    if is_ml and is_lts:
        suffix = 'lts-ml'
    elif is_ml and not is_lts:
        suffix = 'ml'
    elif not is_ml and is_lts:
        suffix = 'lts'
    else:
        suffix = ''
    url = f'https://learn.microsoft.com/en-us/azure/databricks/release-notes/runtime/{version}{suffix}'
    return url


releases_exploded['relative_path'] = releases_exploded.apply(lambda r: build_relative_path(
    r['version'],
    r['is_lts'],
    r['is_ml'],
    r['is_gpu_accelerated']
), axis=1)
releases_exploded['url'] = releases_exploded.apply(lambda r: build_url(
    r['version'],
    r['is_lts'],
    r['is_ml']
), axis=1)


def extract_version(url, xpath):
    arr = extract_from_content2(load_content(url), xpath)
    if len(arr) > 0:
        return arr[0].strip(': ')
    else:
        return None


releases_exploded['scala_version'] = releases_exploded.apply(lambda r: extract_version(r['url'], SCALA_VERSION_XPATH),
                                                             axis=1)
releases_exploded['delta_lake_version'] = releases_exploded.apply(
    lambda r: extract_version(r['url'], DELTA_LAKE_VERSION_XPATH), axis=1)
releases_exploded['scala_version'] = releases_exploded.groupby('version')['scala_version'].transform(
    lambda g: g.ffill().bfill())
releases_exploded['delta_lake_version'] = releases_exploded.groupby('version')['delta_lake_version'].transform(
    lambda g: g.ffill().bfill())

In [32]:
releases_exploded[
    ['version', 'is_lts', 'is_ml', 'is_gpu_accelerated', 'spark_version', 'scala_version', 'delta_lake_version',
     'release_date', 'end_of_support_date',
     'relative_path', 'url']].sort_values(by=['release_date', 'is_lts', 'is_ml', 'is_gpu_accelerated']).to_parquet(
    'releases.parquet', index=False)

In [37]:
poms_configurations = releases_exploded[['version', 'is_lts', 'is_ml', 'is_gpu_accelerated', 'spark_version', 'url']]

installed_libraries = []

for index, pc in poms_configurations.iterrows():
    url = pc['url']
    print('Current configuration version: ', pc['version'], 'is_ml', pc['is_ml'])
    if not pc['is_ml']:
        table = load_table(url, INSTALLED_JAVA_SCALA_LIBS_XPATH)
    else:
        if pc['is_gpu_accelerated']:
            table = load_table(url, INSTALLED_JAVA_SCALA_LIBS_ML_GPU_XPATH)
        else:
            table = load_table(url, INSTALLED_JAVA_SCALA_LIBS_ML_CPU_XPATH)
    table_renamed = table.rename(columns={
        'Group ID': 'group_id',
        'Artifact ID': 'artifact_id',
        'Version': 'artifact_version'})
    table_renamed[[
        'version',
        'is_lts',
        'is_ml',
        'is_gpu_accelerated'
    ]] = pc.loc[[
        'version',
        'is_lts',
        'is_ml',
        'is_gpu_accelerated'
    ]].values  # TODO: think of a better way
    installed_libraries.append(table_renamed)

installed_libraries = pd.concat(installed_libraries, ignore_index=True)

Current configuration version:  15.2 is_ml True
Current configuration version:  15.2 is_ml True
Current configuration version:  15.2 is_ml False
Current configuration version:  15.1 is_ml True
Current configuration version:  15.1 is_ml True
Current configuration version:  15.1 is_ml False
Current configuration version:  14.3 is_ml True
Current configuration version:  14.3 is_ml True
Current configuration version:  14.3 is_ml False
Current configuration version:  14.2 is_ml True
Current configuration version:  14.2 is_ml True
Current configuration version:  14.2 is_ml False
Current configuration version:  14.1 is_ml True
Current configuration version:  14.1 is_ml True
Current configuration version:  14.1 is_ml False
Current configuration version:  13.3 is_ml True
Current configuration version:  13.3 is_ml True
Current configuration version:  13.3 is_ml False
Current configuration version:  12.2 is_ml True
Current configuration version:  12.2 is_ml True
Current configuration version:  12

In [38]:
installed_libraries[
    ['version', 'is_lts', 'is_ml', 'is_gpu_accelerated', 'group_id', 'artifact_id', 'artifact_version']].sort_values(
    by=['version', 'is_lts', 'is_ml', 'is_gpu_accelerated', 'group_id', 'artifact_id']).to_parquet(
    'installed_libraries.parquet', index=False)